In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv("2021_12_07.csv", parse_dates = ["start_date", "end_date"], dtype={'work_postal':'str'})
df2 = pd.read_csv("comprate_2021-12-07-10-26-31.csv")

#Replace under_29 by .under_29 to make it easier when sorting
df['age_group'] = df['age_group'].replace(to_replace='under_29', value='.under_29')

#Fill in na values for age_group
df['age_group'] = df['age_group'].fillna(df['age_group'].value_counts().index[0])

#Replace missing values in event column with unknown
df['event'] = df['event'].fillna('unknown')

In [2]:
#Temporary end_date: this end date is used if the employee is still working
temp_end_date = pd.to_datetime('2021-12-07')

#Get the list of all employees by their unique IDs
employee_ids = df.emplid_sec.unique()

#Define list of features we want in our model
duration = []
division = []
department = []
comprate = []
last_pay_raise = []
highest_educ_lvl = []
age_group = []
pay_increase_ot = []
last_jobtitle_duration = []
piot_compared_avg = [] #pay increase over time compared with average
event = []

In [3]:
#Loop through each employee records
for ID in employee_ids:
    #Get all records of the employee
    employee = df[df['emplid_sec'] == ID].copy()
    
    ##### DURATION #####
    #number of days worked as of 2021-12-07 (Includes end date)
    duration.append(int(sum(employee['duration'].tolist(), employee.shape[0])))
    
    ##### DIVISION #####
    #Get the last division they were in
    employee.sort_values(by=['end_date'], inplace=True)
    division.append(employee.iloc[-1]['division'])
    
    ##### DEPARTMENT #####
    #Get the last department they were in
    employee.sort_values(by=['end_date'], inplace=True)
    department.append(employee.iloc[-1]['department'])
    
    ##### COMP RATE #####
    #Get their highest comprate
    comprate.append(max(employee['comprate'].tolist()))
    
    ##### LAST PAY RAISE #####
    #Get last date of work or temporary last date
    if(employee['end_date'].isna().sum()):
        end = temp_end_date
    else:
        end = employee['end_date'].sort_values().tolist()[-1]
    #Get date of last pay raise
    employee.sort_values(by=['comprate'], inplace=True)
    last_raise = employee.iloc[-1]['start_date']
    #Calculate the difference
    last_pay_raise.append((end - last_raise).days)
    
    ##### EDUCATION LEVEL #####
    #Get the highest education level
    highest_educ_lvl.append(sorted(employee['highest_educ_lvl'].tolist())[-1])
    
    ##### AGE GROUP #####
    #Get the age group they were before they left
    age_group.append(sorted(employee['age_group'].tolist())[-1])
    
    ##### COMPRATE INCREASE OVER TIME #####
    #(max - min) / duration
    max_rate = max(employee['comprate'].tolist())
    min_rate = min(employee['comprate'].tolist())
    pay_increase_ot.append((max_rate - min_rate) / duration[-1])
    
    ##### DURATION OF CURRENT POSITION #####
    #Get the duration in days of the last jobtitle they held or currently holding
    employee.sort_values(by=['end_date'], inplace=True)
    last_jobtitle = employee.iloc[-1]['jobtitle']
    last_jobtitle_duration.append(employee.iloc[-1]['duration'] + 1)
    for i in reversed(range(len(employee) - 1)):
        if(employee.iloc[i]['jobtitle'] == last_jobtitle):
            last_jobtitle_duration[-1] += employee.iloc[i]['duration'] + 1
        else:
            break
    
    ##### EVENT #####
    #Get the employee's latest event
    #Unknown, Retirement, Termination
    employee.sort_values(by=['end_date'], inplace=True)
    event.append(employee.iloc[-1]['event'])

In [4]:
#Get the average comprate increase over time
avg_pay_increase_ot = sum(pay_increase_ot) / len(pay_increase_ot)

##### COMPRATE INCREASE OVER TIME COMPARED TO AVERAGE #####
#calculate the % below or above average
for val in pay_increase_ot:
    piot_compared_avg.append((val - avg_pay_increase_ot) / avg_pay_increase_ot)

In [5]:
##### HIGHEST JOBTITLE --WORK IN PROGRESS-- #####
#Current technique: calculate the average comprate for each jobtitle and sort by the average for its rank

#Get all current jobtitles
jobtitles = df['jobtitle'].unique()

#List to store average
pay_avg_by_jobtitle = []

#Loop through each jobtitles
for jt in jobtitles:
    #Get all records with the current jobtitle
    records = df[df['jobtitle'] == jt].copy()
    #Get the maximum comprate for each employee
    max_comprates_by_jt = records.groupby(['emplid_sec'])['comprate'].max()
    #Calculate the average
    pay_avg_by_jobtitle = sum(max_comprates_by_jt) / len(max_comprates_by_jt)

In [6]:
### For One-Hot Encoding
# X = pd.get_dummies(X, prefix = ['division', 'department', ...], columns = ['division', 'department', ...])

### For Label Encoding
# from sklearn.preprocessing import LabelEncoder
# y = LabelEncoder().fit_transform(y)
# substitute y with whatever column you want to encode